In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Alagoas - Consumo de Cimento (t)'
start_index = 60
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [7]:
data = pd.read_csv('2003_mo_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Alagoas - IDH,Alagoas - IDH Longevidade,Alagoas - value,Alagoas - IDH Renda,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - Consumo de Cimento (t)
0,2003-1,0.635160,0.728082,0.294434,0.587982,8.765721,30.779440,2.590528e+07,1.303625e+06,7.576443,...,1.921789e+10,8.202881e+10,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,30.109
1,2003-2,0.635571,0.728532,0.296211,0.588354,8.760196,30.599132,2.593685e+07,1.305020e+06,7.578654,...,1.921789e+10,8.222716e+10,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,27.684
2,2003-3,0.635982,0.728982,0.297521,0.588725,8.754671,30.545910,2.596843e+07,1.306414e+06,7.580866,...,1.921789e+10,8.234287e+10,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,26.534
3,2003-4,0.636393,0.729432,0.298766,0.589097,8.749145,30.377624,2.600000e+07,1.307809e+06,7.583078,...,1.921789e+10,8.235526e+10,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,25.109
4,2003-5,0.636804,0.729882,0.299945,0.589469,8.743620,30.464900,2.603157e+07,1.309203e+06,7.585289,...,1.921843e+10,8.245980e+10,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,25.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2022-9,NaN,NaN,0.587084,0.629024,NaN,24.501314,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.781
237,2022-10,NaN,NaN,0.585891,0.629017,NaN,24.609100,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.171
238,2022-11,NaN,NaN,0.584366,0.629010,NaN,24.538488,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.126
239,2022-12,NaN,NaN,0.582450,0.629003,NaN,24.688196,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.126


In [8]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Alagoas - IDH,Alagoas - IDH Longevidade,Alagoas - value,Alagoas - IDH Renda,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.146764,-2.407288,-1.449375,-2.067761,-0.937577,-0.789529,-1.703997,-1.236973,-1.975005,-1.922747,...,1.155957,-1.063593,-0.822933,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-2.113412,-2.364085,-1.423792,-2.040187,-0.939615,-0.803493,-1.687205,-1.181428,-1.949747,-1.897135,...,1.155957,-1.063593,-0.818174,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-2.080060,-2.320881,-1.404924,-2.012614,-0.941652,-0.807615,-1.670414,-1.125883,-1.924488,-1.871523,...,1.155957,-1.063593,-0.815398,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-2.046707,-2.277677,-1.386985,-1.985041,-0.943690,-0.820649,-1.653622,-1.070338,-1.899230,-1.845911,...,1.155957,-1.063593,-0.815101,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-2.013355,-2.234474,-1.370013,-1.957467,-0.945728,-0.813889,-1.636831,-1.014793,-1.873971,-1.820299,...,1.151943,-1.063566,-0.812592,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.183322,0.868238,1.308074,1.131052,0.919145,-1.198723,0.963258,-0.926262,0.929504,0.848917,...,3.115817,1.690001,0.667867,-0.689886,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.177023,0.846976,1.359203,1.127894,0.913671,-1.181757,0.946383,-0.898324,0.909729,0.830432,...,3.295619,1.717926,0.640959,-0.441954,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.170724,0.825714,1.405650,1.124737,0.908197,-1.171317,0.929508,-0.870386,0.889955,0.811947,...,3.486686,1.748758,0.615514,-0.132782,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.164425,0.804452,1.459075,1.121579,0.902724,-1.180748,0.912633,-0.842448,0.870180,0.793463,...,3.692016,1.780444,0.593546,0.084061,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      27.684
1      26.534
2      25.109
3      25.171
4      22.645
        ...  
236    51.171
237    49.126
238    49.126
239       NaN
240       NaN
Name: Alagoas - Consumo de Cimento (t), Length: 241, dtype: float64

In [10]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Alagoas - IDH,Alagoas - IDH Longevidade,Alagoas - value,Alagoas - IDH Renda,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
60,-0.508441,-0.376990,-0.733600,-0.605124,-1.019703,-0.037973,-0.848808,1.405264,-0.742184,-0.808534,...,-0.660781,-0.907428,-1.023454,-0.537241,0.042292,0.488894,-0.717137,-0.601292,-0.315948,0.240813
61,-0.489848,-0.356716,-0.701180,-0.584972,-1.018834,-0.062281,-0.840179,1.413569,-0.737145,-0.804966,...,-0.660781,-0.899853,-1.024085,-0.479316,0.011943,-0.072763,-0.714911,-0.580436,-0.286095,0.246566
62,-0.471254,-0.336442,-0.668084,-0.564820,-1.017965,-0.050358,-0.831550,1.421875,-0.732105,-0.801398,...,-0.660781,-0.892039,-1.024307,-0.466312,-0.019339,-0.316940,-0.694168,-0.559580,-0.256243,0.315327
63,-0.452661,-0.316167,-0.646235,-0.544669,-1.017096,-0.050171,-0.822921,1.430181,-0.727066,-0.797829,...,-0.660781,-0.883981,-1.024104,-0.438778,-0.059264,0.180777,-0.700209,-0.538723,-0.226390,0.356189
64,-0.434068,-0.295893,-0.619607,-0.524517,-1.016226,-0.021423,-0.814292,1.438487,-0.722026,-0.794261,...,-0.660781,-0.875677,-1.023461,-0.457839,-0.104671,-0.212484,-0.660717,-0.517867,-0.196537,0.524165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.324514,1.370260,1.121339,1.196483,1.186919,-1.069189,1.177341,-1.214407,1.097518,1.109211,...,-0.660781,1.272681,1.521503,-0.742853,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.323316,1.364348,1.103076,1.194330,1.173733,-1.161695,1.181165,-1.230415,1.105489,1.111823,...,-0.660781,1.288866,1.512161,-0.787367,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.322118,1.358436,1.083984,1.192177,1.160546,-1.230868,1.184990,-1.246424,1.113460,1.114436,...,-0.660781,1.304998,1.500026,-0.757940,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.320921,1.352525,1.070055,1.190024,1.147360,-1.298996,1.188814,-1.262433,1.121431,1.117048,...,-0.660781,1.332157,1.486995,-0.917469,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

60     32.977000
61     35.662000
62     39.980000
63     34.089000
64     32.715000
         ...    
163    42.491371
164    41.535029
165    40.578686
166    39.622343
167    38.666000
Name: Alagoas - Consumo de Cimento (t), Length: 108, dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Alagoas - IDH,Alagoas - IDH Longevidade,Alagoas - value,Alagoas - IDH Renda,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
168,1.318526,1.340702,1.040654,1.185718,1.120987,-1.422705,1.196462,-1.294450,1.137372,1.122273,...,-0.660781,1.364625,1.460212,-1.032458,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117
169,1.317834,1.329890,1.028253,1.182806,1.113532,-1.481842,1.193806,-1.294397,1.136752,1.115681,...,-0.660781,1.371257,1.445627,-1.077854,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274
170,1.317142,1.319078,1.015519,1.179895,1.106076,-1.530176,1.191149,-1.294344,1.136131,1.109088,...,-0.660781,1.375328,1.430008,-1.036724,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341
171,1.316450,1.308266,1.002465,1.176984,1.098621,-1.593615,1.188492,-1.294290,1.135511,1.102495,...,-0.660781,1.381257,1.413730,-1.013400,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301
172,1.315758,1.297455,0.986107,1.174073,1.091166,-1.643819,1.185835,-1.294237,1.134890,1.095903,...,-0.660781,1.364882,1.396980,-1.059228,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284
173,1.315066,1.286643,0.978396,1.171162,1.083710,-1.698151,1.183179,-1.294184,1.134270,1.089310,...,-0.582593,1.369812,1.379116,-1.170032,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348
174,1.314374,1.275831,0.970719,1.168251,1.076255,-1.715076,1.180522,-1.294131,1.133650,1.082718,...,-0.474908,1.376543,1.360141,-1.115621,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659
175,1.313682,1.265019,0.963104,1.165340,1.068800,-1.713343,1.177865,-1.294077,1.133029,1.076125,...,-0.364364,1.384496,1.341361,-1.182159,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184
176,1.312990,1.254208,0.955578,1.162429,1.061345,-1.705266,1.175209,-1.294024,1.132409,1.069532,...,-0.250935,1.391358,1.323058,-1.270623,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173
177,1.312298,1.243396,0.948164,1.159518,1.053889,-1.685174,1.172552,-1.293971,1.131788,1.062940,...,-0.134596,1.399740,1.303196,-1.321931,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168    32.321
169    41.937
170    31.982
171    28.499
172    22.626
173    21.340
174    26.848
175    28.074
176    29.941
177    28.977
178    31.580
179    29.991
180    26.820
181    29.446
182    27.666
183    22.013
184    28.025
185    25.648
186    31.782
187    27.768
188    33.073
189    30.389
190    27.657
191    32.388
192    30.378
193    30.641
194    32.337
195    36.405
196    22.499
197    27.961
198    32.378
199    32.582
200    42.703
201    38.584
202    36.503
203    43.323
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [14]:
def validation_splitter(df, div_factor, add_factor):
    split_factor = len(df)//div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop_index.append(pos + add_factor)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop_index), df.iloc[positions_to_drop]

In [18]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 10, 60)
    target,target_val = validation_splitter(train_target, 10, 60)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2667793495, 3679075897, 699231315, 3981700790, 3993833824, 611241559, 1073349128, 2591698476, 3985686111, 1222666869, 3417101817, 308071988, 1855621940, 2343980052, 2233326380, 3131319313, 758170537, 1152763993, 4192815503, 132736503, 2005813698, 1317691109, 3359464468, 3158344733, 2015370594]


Step: 0 ___________________________________________
val_loss: 10.611112594604492
winner_seed: 2667793495


Step: 1 ___________________________________________
val_loss: 12.44749641418457


Step: 2 ___________________________________________
val_loss: 16.865930557250977


Step: 3 ___________________________________________
val_loss: 17.789691925048828


Step: 4 ___________________________________________
val_loss: 22.439050674438477


Step: 5 ___________________________________________
val_loss: 6.899697780609131
winner_seed: 611241559


Step: 6 ___________________________________________
val_loss: 14.12260913848877


Step: 7 ___________________________________________
val_loss: 19.094085693359

In [21]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 0s 27ms/step - loss: 2935.4136 - val_loss: 3358.1355
Epoch 2/10000
4/4 [==============================] - 0s 6ms/step - loss: 2079.3257 - val_loss: 4361.6069
Epoch 3/10000
4/4 [==============================] - 0s 6ms/step - loss: 2756.4363 - val_loss: 3616.2195
Epoch 4/10000
4/4 [==============================] - 0s 6ms/step - loss: 1645.2579 - val_loss: 1656.6605
Epoch 5/10000
4/4 [==============================] - 0s 6ms/step - loss: 2364.8628 - val_loss: 455.4429
Epoch 6/10000
4/4 [==============================] - 0s 6ms/step - loss: 1163.5946 - val_loss: 2149.6194
Epoch 7/10000
4/4 [==============================] - 0s 6ms/step - loss: 364.9690 - val_loss: 75.6817
Epoch 8/10000
4/4 [==============================] - 0s 5ms/step - loss: 58.5869 - val_loss: 283.6942
Epoch 9/10000
4/4 [==============================] - 0s 5ms/step - loss: 136.4122 - val_loss: 48.2561
Epoch 10/10000
4/4 [==============================] - 0s 5ms/ste

4/4 [==============================] - 0s 5ms/step - loss: 25.8823 - val_loss: 34.6668
Epoch 82/10000
4/4 [==============================] - 0s 5ms/step - loss: 25.4110 - val_loss: 28.7054
Epoch 83/10000
4/4 [==============================] - 0s 6ms/step - loss: 29.8761 - val_loss: 62.0150
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 53.4259 - val_loss: 32.3053
Epoch 85/10000
4/4 [==============================] - 0s 6ms/step - loss: 24.0225 - val_loss: 29.9659
Epoch 86/10000
4/4 [==============================] - 0s 6ms/step - loss: 22.4444 - val_loss: 38.4908
Epoch 87/10000
4/4 [==============================] - 0s 5ms/step - loss: 39.3272 - val_loss: 32.3943
Epoch 88/10000
4/4 [==============================] - 0s 6ms/step - loss: 33.8005 - val_loss: 28.0860
Epoch 89/10000
4/4 [==============================] - 0s 6ms/step - loss: 33.8427 - val_loss: 39.5136
Epoch 90/10000
4/4 [==============================] - 0s 6ms/step - loss: 31.9898 - val_loss: 49.

Epoch 161/10000
4/4 [==============================] - 0s 6ms/step - loss: 23.3311 - val_loss: 46.3749
Epoch 162/10000
4/4 [==============================] - 0s 6ms/step - loss: 27.5492 - val_loss: 32.4123
Epoch 163/10000
4/4 [==============================] - 0s 6ms/step - loss: 23.7832 - val_loss: 36.5329
Epoch 164/10000
4/4 [==============================] - 0s 6ms/step - loss: 23.8013 - val_loss: 36.4963
Epoch 165/10000
4/4 [==============================] - 0s 6ms/step - loss: 26.6722 - val_loss: 27.8329
Epoch 166/10000
4/4 [==============================] - 0s 5ms/step - loss: 23.4988 - val_loss: 26.8944
Epoch 167/10000
4/4 [==============================] - 0s 6ms/step - loss: 22.4555 - val_loss: 41.9160
Epoch 168/10000
4/4 [==============================] - 0s 5ms/step - loss: 41.8853 - val_loss: 31.2496
Epoch 169/10000
4/4 [==============================] - 0s 6ms/step - loss: 24.4653 - val_loss: 31.8455
Epoch 170/10000
4/4 [==============================] - 0s 5ms/step - loss

4/4 [==============================] - 0s 6ms/step - loss: 38.3534 - val_loss: 62.3707
Epoch 320/10000
4/4 [==============================] - 0s 6ms/step - loss: 39.9869 - val_loss: 52.0111
Epoch 321/10000
4/4 [==============================] - 0s 5ms/step - loss: 30.4478 - val_loss: 29.9171
Epoch 322/10000
4/4 [==============================] - 0s 5ms/step - loss: 23.8872 - val_loss: 26.9481
Epoch 323/10000
4/4 [==============================] - 0s 6ms/step - loss: 23.8125 - val_loss: 29.4594
Epoch 324/10000
4/4 [==============================] - 0s 6ms/step - loss: 22.1746 - val_loss: 26.8960
Epoch 325/10000
4/4 [==============================] - 0s 5ms/step - loss: 21.8061 - val_loss: 31.6534
Epoch 326/10000
4/4 [==============================] - 0s 5ms/step - loss: 24.7490 - val_loss: 28.5626
Epoch 327/10000
4/4 [==============================] - 0s 5ms/step - loss: 19.8008 - val_loss: 42.5115
Epoch 328/10000
4/4 [==============================] - 0s 6ms/step - loss: 28.9159 - val_

4/4 [==============================] - 0s 5ms/step - loss: 22.8961 - val_loss: 28.9941
Epoch 478/10000
4/4 [==============================] - 0s 5ms/step - loss: 21.6286 - val_loss: 28.1266
Epoch 479/10000
4/4 [==============================] - 0s 6ms/step - loss: 19.3571 - val_loss: 49.8669
Epoch 480/10000
4/4 [==============================] - 0s 5ms/step - loss: 27.0404 - val_loss: 31.4938
Epoch 481/10000
4/4 [==============================] - 0s 5ms/step - loss: 20.4679 - val_loss: 28.6787
Epoch 482/10000
4/4 [==============================] - 0s 5ms/step - loss: 20.7475 - val_loss: 57.0883
Epoch 483/10000
4/4 [==============================] - 0s 5ms/step - loss: 27.1023 - val_loss: 29.2453
Epoch 484/10000
4/4 [==============================] - 0s 5ms/step - loss: 20.7722 - val_loss: 30.0711
Epoch 485/10000
4/4 [==============================] - 0s 5ms/step - loss: 19.8239 - val_loss: 30.1268
Epoch 486/10000
4/4 [==============================] - 0s 5ms/step - loss: 20.7098 - val_

...

Epoch 1659/10000
4/4 [==============================] - 0s 6ms/step - loss: 36.6928 - val_loss: 30.2442
Epoch 1660/10000
4/4 [==============================] - 0s 6ms/step - loss: 44.5374 - val_loss: 24.2725
Epoch 1661/10000
4/4 [==============================] - 0s 5ms/step - loss: 33.8558 - val_loss: 31.7531
Epoch 1662/10000
4/4 [==============================] - 0s 5ms/step - loss: 39.5014 - val_loss: 33.6612
Epoch 1663/10000
4/4 [==============================] - 0s 5ms/step - loss: 29.0407 - val_loss: 42.4150
Epoch 1664/10000
4/4 [==============================] - 0s 5ms/step - loss: 28.4688 - val_loss: 28.6351
Epoch 1665/10000
4/4 [==============================] - 0s 5ms/step - loss: 24.0476 - val_loss: 35.1000
Epoch 1666/10000
4/4 [==============================] - 0s 5ms/step - loss: 25.4777 - val_loss: 76.7630
Epoch 1667/10000
4/4 [==============================] - 0s 5ms/step - loss: 33.4164 - val_loss: 36.3319
Epoch 1668/10000
4/4 [==============================] - 0s 5ms/s

Epoch 1738/10000
4/4 [==============================] - 0s 5ms/step - loss: 37.5761 - val_loss: 22.5570
Epoch 1739/10000
4/4 [==============================] - 0s 5ms/step - loss: 32.9132 - val_loss: 24.0733
Epoch 1740/10000
4/4 [==============================] - 0s 5ms/step - loss: 31.1498 - val_loss: 25.6381
Epoch 1741/10000
4/4 [==============================] - 0s 5ms/step - loss: 32.1265 - val_loss: 32.2171
Epoch 1742/10000
4/4 [==============================] - 0s 5ms/step - loss: 30.7524 - val_loss: 20.2265
Epoch 1743/10000
4/4 [==============================] - 0s 5ms/step - loss: 30.2497 - val_loss: 16.2490
Epoch 1744/10000
4/4 [==============================] - 0s 5ms/step - loss: 27.7946 - val_loss: 23.6858
Epoch 1745/10000
4/4 [==============================] - 0s 5ms/step - loss: 27.9332 - val_loss: 19.0606
Epoch 1746/10000
4/4 [==============================] - 0s 5ms/step - loss: 30.7119 - val_loss: 35.5569
Epoch 1747/10000
4/4 [==============================] - 0s 5ms/s

Epoch 1895/10000
4/4 [==============================] - 0s 5ms/step - loss: 40.0864 - val_loss: 20.5978
Epoch 1896/10000
4/4 [==============================] - 0s 5ms/step - loss: 29.7062 - val_loss: 14.4957
Epoch 1897/10000
4/4 [==============================] - 0s 5ms/step - loss: 25.1457 - val_loss: 25.5906
Epoch 1898/10000
4/4 [==============================] - 0s 5ms/step - loss: 27.0523 - val_loss: 20.3183
Epoch 1899/10000
4/4 [==============================] - 0s 5ms/step - loss: 22.8196 - val_loss: 31.9846
Epoch 1900/10000
4/4 [==============================] - 0s 5ms/step - loss: 30.7233 - val_loss: 29.3609
Epoch 1901/10000
4/4 [==============================] - 0s 5ms/step - loss: 23.1754 - val_loss: 24.3544
Epoch 1902/10000
4/4 [==============================] - 0s 5ms/step - loss: 25.7610 - val_loss: 18.4019
Epoch 1903/10000
4/4 [==============================] - 0s 5ms/step - loss: 23.4688 - val_loss: 38.1918
Epoch 1904/10000
4/4 [==============================] - 0s 5ms/s

Epoch 1974/10000
4/4 [==============================] - 0s 5ms/step - loss: 39.6046 - val_loss: 27.6412
Epoch 1975/10000
4/4 [==============================] - 0s 5ms/step - loss: 36.0874 - val_loss: 27.3128
Epoch 1976/10000
4/4 [==============================] - 0s 5ms/step - loss: 30.0926 - val_loss: 26.2190
Epoch 1977/10000
4/4 [==============================] - 0s 5ms/step - loss: 32.1694 - val_loss: 73.5114
Epoch 1978/10000
4/4 [==============================] - 0s 5ms/step - loss: 70.4316 - val_loss: 33.2992
Epoch 1979/10000
4/4 [==============================] - 0s 5ms/step - loss: 33.3472 - val_loss: 21.9726
Epoch 1980/10000
4/4 [==============================] - 0s 5ms/step - loss: 34.8038 - val_loss: 35.5420
Epoch 1981/10000
4/4 [==============================] - 0s 5ms/step - loss: 38.8439 - val_loss: 22.6124
Epoch 1982/10000
4/4 [==============================] - 0s 5ms/step - loss: 29.7226 - val_loss: 25.9689
Epoch 1983/10000
4/4 [==============================] - 0s 5ms/s

Epoch 2131/10000
4/4 [==============================] - 0s 5ms/step - loss: 31.5440 - val_loss: 44.5841
Epoch 2132/10000
4/4 [==============================] - 0s 5ms/step - loss: 24.9326 - val_loss: 43.8638
Epoch 2133/10000
4/4 [==============================] - 0s 5ms/step - loss: 22.6937 - val_loss: 36.9791
Epoch 2134/10000
4/4 [==============================] - 0s 5ms/step - loss: 21.3504 - val_loss: 28.2143
Epoch 2135/10000
4/4 [==============================] - 0s 5ms/step - loss: 34.3895 - val_loss: 25.3517
Epoch 2136/10000
4/4 [==============================] - 0s 5ms/step - loss: 24.2897 - val_loss: 25.9107
Epoch 2137/10000
4/4 [==============================] - 0s 6ms/step - loss: 24.7701 - val_loss: 29.9247
Epoch 2138/10000
4/4 [==============================] - 0s 5ms/step - loss: 22.9938 - val_loss: 29.7809
Epoch 2139/10000
4/4 [==============================] - 0s 5ms/step - loss: 23.9791 - val_loss: 74.1478
Epoch 2140/10000
4/4 [==============================] - 0s 5ms/s

In [22]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = mae_mape_calculator(trained_model, test_input, test_target, 168)

1/1 [==============================] - 0s 49ms/step
Month-1 - Error: [[10.011406]]
1/1 [==============================] - 0s 14ms/step
Month-2 - Error: [[1.2752647]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[6.5260563]]
1/1 [==============================] - 0s 13ms/step
Month-4 - Error: [[1.0466137]]
1/1 [==============================] - 0s 13ms/step
Month-5 - Error: [[32.4333]]
1/1 [==============================] - 0s 13ms/step
Month-6 - Error: [[38.91546]]
1/1 [==============================] - 0s 13ms/step
Month-7 - Error: [[45.640312]]
1/1 [==============================] - 0s 13ms/step
Month-8 - Error: [[46.867416]]
1/1 [==============================] - 0s 13ms/step
Month-9 - Error: [[48.734478]]
1/1 [==============================] - 0s 13ms/step
Month-10 - Error: [[47.770485]]
1/1 [==============================] - 0s 13ms/step
Month-11 - Error: [[50.373486]]
1/1 [==============================] - 0s 14ms/step
Month-12 - Error: [[48.784485]]
1/1 

In [24]:
display(mae)
display(mape)

46.08247

1.5300299

In [25]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [26]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[8.8121605]] - Target[354.11599999999993]| =  Error: [[345.30383]]; MAPE:[[0.97511506]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[-243.5524]] - Target[342.67499999999995]| =  Error: [[586.2274]]; MAPE:[[1.7107389]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[-288.06888]] - Target[406.294]| =  Error: [[694.3629]]; MAPE:[[1.7090158]]


[array([[345.30383]], dtype=float32),
 array([[586.2274]], dtype=float32),
 array([[694.3629]], dtype=float32)]

541.9647

1.4649566